In [ ]:
#@markdown
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import httplib2
import time
import json
import base64
import glob
import os
from os import walk
from urllib import parse
import math

#from google.colab import drive
#drive.mount('/content/drive')

#config
pd.set_option('display.max_columns', None)

In [ ]:


#@markdown #***INGRESO DE USUARIO***
Dominio = '' #@param {type: "string"}
Email = '' #@param {type:"string"}
Token = ''#@param {type:"string"}

z_domain= Dominio
email= Email
contraseña= Token


from tqdm import tqdm
from time import sleep

for i in tqdm(range(10)):
  sleep(.7)
authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')

headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}
start_time="946684800"



In [ ]:
#@markdown
qw= True
all_df=pd.DataFrame()
url = z_domain+"/api/v2/users/search?query=role:agent%20role:admin"
while(qw):
  print(url)
  http = httplib2.Http()
  response, content = http.request(url, 'GET', headers=headers)
  data=json.loads(content)
  print(data)
  url=data['next_page']
  df=pd.DataFrame(data['users'])
  all_df=pd.concat([all_df,df[['name','email','created_at','updated_at','last_login_at','custom_role_id']]])
  try:
    count=math.ceil(data['count']/100)
    num=np.array(parse.parse_qs(parse.urlsplit(url).query)['page'])
    if count>=int(num):
      print("Valido",count,"Es mayor que",int(num))
      qw= True
    else:
      qw=False
  except:
    qw= False




In [ ]:
#@markdown
all_df

In [ ]:
#@markdown
url2 = z_domain+"/api/v2/custom_roles"
http2 = httplib2.Http()
response, content = http2.request(url2, 'GET', headers=headers)
data2=json.loads(content)
roles=pd.DataFrame(data2['custom_roles'])
roles['name'][roles['id']==360010165171]

In [ ]:
#@markdown *Generando csv*
lst=[]
for i in all_df['custom_role_id']:
  lst.append(*list(roles['name'][roles['id']==i]))
all_df['Nombre del rol']= lst

all_df=all_df.reset_index()
all_df.to_csv("extracto.csv")

from tqdm import tqdm
from time import sleep

for i in tqdm(range(10)):
  sleep(.7)
print("CSV generado con exito")